In [16]:
!pip install bs4
!pip install lxml

from bs4 import BeautifulSoup
import pandas as pd
import requests
print('bs4 and lxml installed')

bs4 and lxml installed


# Scraping the wikipedia page of Canada's postal codes

In [25]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'html.parser')
table = soup.find('table')

In [26]:
list_columns = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = list_columns)

In [28]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [29]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Cleaning the data

In [33]:
df = df[ df['Borough']!= 'Not assigned']

In [41]:
df = df[df['Neighborhood']!= 'Not assigned']

In [42]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [43]:
temp_df = df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ','.join(x))
temp_df = temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'}, inplace=True)

In [44]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [45]:
df_merge.drop(['Neighborhood'], axis=1, inplace=True)

In [46]:
df_merge.drop_duplicates(inplace=True)

In [47]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'}, inplace=True)

In [48]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [49]:
df_merge.shape

(103, 3)